# Table of Contents:
* [Data Cleaning and Exploratory Analysis](#first-bullet)
* [Preprocessing](#second-bullet)
* [Modelling](#third-bullet)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Data Cleaning and Exploratory Analysis <a class="anchor" id="first-bullet"></a>

In [2]:
# loading in the data
companies = pd.read_csv('companies.csv',sep=';')

In [3]:
# surveying
companies.head()

,Name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,permalink
0,#fame,http://livfame.com,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,05/01/2015,05/01/2015,/organization/-fame
1,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,04/09/2014,01/03/2014,14/10/2014,/organization/-qounter
2,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,30/01/2014,30/01/2014,/organization/-the-one-of-them-inc-
3,0-6.com,http://www.0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,01/01/2007,19/03/2008,19/03/2008,/organization/0-6-com
4,004 Technologies,http://004gmbh.de/en/004-interact,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,01/01/2010,24/07/2014,24/07/2014,/organization/004-technologies


In [4]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63215 entries, 0 to 63214
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               63214 non-null  object
 1   homepage_url       59069 non-null  object
 2   category_list      63215 non-null  object
 3   funding_total_usd  63215 non-null  object
 4   status             63215 non-null  object
 5   country_code       57799 non-null  object
 6   state_code         56263 non-null  object
 7   region             56760 non-null  object
 8   city               56762 non-null  object
 9   funding_rounds     63215 non-null  int64 
 10  founded_at         49707 non-null  object
 11  first_funding_at   63192 non-null  object
 12  last_funding_at    63215 non-null  object
 13  permalink          63215 non-null  object
dtypes: int64(1), object(13)
memory usage: 6.8+ MB


In [5]:
companies.isna().sum()

Name                     1
homepage_url          4146
category_list            0
funding_total_usd        0
status                   0
country_code          5416
state_code            6952
region                6455
city                  6453
funding_rounds           0
founded_at           13508
first_funding_at        23
last_funding_at          0
permalink                0
dtype: int64

In [6]:
# checking the row with a missing name
companies[companies['Name'].isna()]

,Name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,permalink
54191,NaN,http://tellitin10.com,Startups,25000,closed,USA,OR,"Portland, Oregon",Portland,1,01/10/2011,01/03/2012,01/03/2012,/organization/tell-it-in


In [7]:
# just drop that one row
companies.dropna(subset='Name', inplace=True)

In [8]:
# row is gone
companies.shape

(63214, 14)

In [9]:
# dropping URL column because I'm not sure how it would fit into a beginning model
companies.drop(columns='homepage_url', inplace=True)

In [10]:
# dropping region column because it seems redundant with city column
companies.drop(columns='region', inplace=True)

In [11]:
# dropping permalink column because it's a reference to crunchbase URLs which aren't relevant here
companies.drop(columns='permalink', inplace=True)

In [12]:
# dropping the rows where any of the location columns are null
companies.dropna(subset=['country_code', 'state_code', 'city'], inplace=True)

In [13]:
companies.head()

,Name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,#fame,Media,10000000,operating,IND,16,Mumbai,1,NaN,05/01/2015,05/01/2015
1,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,04/09/2014,01/03/2014,14/10/2014
3,0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,1,01/01/2007,19/03/2008,19/03/2008
4,004 Technologies,Software,-,operating,USA,IL,Champaign,1,01/01/2010,24/07/2014,24/07/2014
6,Ondine Biomedical Inc.,Biotechnology,762851,operating,CAN,BC,Vancouver,2,01/01/1997,11/09/2009,21/12/2009


In [14]:
# resetting the index since dropping a lot of rows messed it up
companies.reset_index(drop=True, inplace=True)

In [15]:
companies.isna().sum()

Name                     0
category_list            0
funding_total_usd        0
status                   0
country_code             0
state_code               0
city                     0
funding_rounds           0
founded_at           10709
first_funding_at        22
last_funding_at          0
dtype: int64

Now the only null values left are with the 'founded_at' and 'first_funding_at' columns. I want to check if the dates from these columns coincide at all for most companies so that I can impute missing values for 'founded_at' from 'first_funding_at' (and vice versa). Before doing that I will need to convert these date columns to datetime types since they are currently objects.

In [16]:
# I ran all the subsequent cells in the process of figuring out how to change the column to datetime
# they are commented out because they all returned errors and I wanted to be able to run all cells in the notebook automatically

In [17]:
#companies['founded_at'] = pd.to_datetime(companies['founded_at'])

In [18]:
# trying again, but with the format argument set to 'mixed' in accordance with the error's advice
#companies['founded_at'] = pd.to_datetime(companies['founded_at'], format='mixed')

In [19]:
# the error says there is a timestamp out of bounds, which company is this?
companies[companies['founded_at'] == '1636-09-08']

,Name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
21034,Harvard University,Colleges|Education|Universities,90000000,operating,USA,MA,Cambridge,1,1636-09-08,06/01/2014,06/01/2014


In [20]:
# I'll remove this row (since Harvard isn't exactly a startup anyway), and then retry converting to datetime
companies.loc[21034,:]

Name                              Harvard University
category_list        Colleges|Education|Universities
funding_total_usd                           90000000
status                                     operating
country_code                                     USA
state_code                                        MA
city                                       Cambridge
funding_rounds                                     1
founded_at                                1636-09-08
first_funding_at                          06/01/2014
last_funding_at                           06/01/2014
Name: 21034, dtype: object

In [21]:
companies.drop(21034, inplace=True)

In [22]:
companies.reset_index(drop=True, inplace=True)

In [23]:
companies.loc[21034,:] #it's gone

Name                                                           Harvest
category_list        Billing|Enterprise Software|Project Management...
funding_total_usd                                                    -
status                                                       operating
country_code                                                       USA
state_code                                                          NY
city                                                          New York
funding_rounds                                                       1
founded_at                                                  01/04/2006
first_funding_at                                            01/01/2008
last_funding_at                                             01/01/2008
Name: 21034, dtype: object

In [24]:
# let's try converting to datetime again
companies['founded_at'] = pd.to_datetime(companies['founded_at'], format='mixed')

In [25]:
companies.info() # success!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55975 entries, 0 to 55974
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               55975 non-null  object        
 1   category_list      55975 non-null  object        
 2   funding_total_usd  55975 non-null  object        
 3   status             55975 non-null  object        
 4   country_code       55975 non-null  object        
 5   state_code         55975 non-null  object        
 6   city               55975 non-null  object        
 7   funding_rounds     55975 non-null  int64         
 8   founded_at         45266 non-null  datetime64[ns]
 9   first_funding_at   55953 non-null  object        
 10  last_funding_at    55975 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 4.7+ MB


In [26]:
companies.head() # nan changed to NaT

,Name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,#fame,Media,10000000,operating,IND,16,Mumbai,1,NaT,05/01/2015,05/01/2015
1,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,2014-04-09,01/03/2014,14/10/2014
2,0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,1,2007-01-01,19/03/2008,19/03/2008
3,004 Technologies,Software,-,operating,USA,IL,Champaign,1,2010-01-01,24/07/2014,24/07/2014
4,Ondine Biomedical Inc.,Biotechnology,762851,operating,CAN,BC,Vancouver,2,1997-01-01,11/09/2009,21/12/2009


In [27]:
# let's try the same for first_funding_at
#companies['first_funding_at'] = pd.to_datetime(companies['first_funding_at'], format='mixed')

In [28]:
# another out of bounds timestamp, which one is this?
companies.loc[6304]

Name                                                  Blaze Bioscience
category_list        Biotechnology|Health Care|Health Care Informat...
funding_total_usd                                             32936998
status                                                       operating
country_code                                                       USA
state_code                                                          WA
city                                                           Seattle
funding_rounds                                                       5
founded_at                                         2010-01-01 00:00:00
first_funding_at                                            1532-06-26
last_funding_at                                             03/12/2013
Name: 6304, dtype: object

In [29]:
# this is clearly an erroneous date, but I don't want to drop the row. I'll change this value to nan
companies.at[6304, 'first_funding_at'] = np.nan

In [30]:
companies.loc[6304] # it's been changed

Name                                                  Blaze Bioscience
category_list        Biotechnology|Health Care|Health Care Informat...
funding_total_usd                                             32936998
status                                                       operating
country_code                                                       USA
state_code                                                          WA
city                                                           Seattle
funding_rounds                                                       5
founded_at                                         2010-01-01 00:00:00
first_funding_at                                                   NaN
last_funding_at                                             03/12/2013
Name: 6304, dtype: object

In [31]:
# try again
#companies['first_funding_at'] = pd.to_datetime(companies['first_funding_at'], format='mixed')

In [32]:
# another out of bounds timestamp
companies.loc[18833]

Name                                                         Gamewheel
category_list        Brand Marketing|Game|Mobile Advertising|Social...
funding_total_usd                                                    -
status                                                       operating
country_code                                                       DEU
state_code                                                          16
city                                                            Berlin
funding_rounds                                                       2
founded_at                                                         NaT
first_funding_at                                            1201-11-23
last_funding_at                                             03/12/2014
Name: 18833, dtype: object

In [33]:
# this also seems like a mess up, change it to nan as well
companies.at[18833, 'first_funding_at'] = np.nan

In [34]:
companies.loc[18833]

Name                                                         Gamewheel
category_list        Brand Marketing|Game|Mobile Advertising|Social...
funding_total_usd                                                    -
status                                                       operating
country_code                                                       DEU
state_code                                                          16
city                                                            Berlin
funding_rounds                                                       2
founded_at                                                         NaT
first_funding_at                                                   NaN
last_funding_at                                             03/12/2014
Name: 18833, dtype: object

In [35]:
# and again
#companies['first_funding_at'] = pd.to_datetime(companies['first_funding_at'], format='mixed')

In [36]:
# another out of bounds timestamp
companies.loc[31357]

Name                             Mousera
category_list              Biotechnology
funding_total_usd               29940004
status                         operating
country_code                         USA
state_code                            CA
city                           San Mateo
funding_rounds                         3
founded_at           2013-01-01 00:00:00
first_funding_at              1014-03-01
last_funding_at               09/06/2015
Name: 31357, dtype: object

In [37]:
# change this to nan as well
companies.at[31357, 'first_funding_at'] = np.nan

In [38]:
companies.loc[31357]

Name                             Mousera
category_list              Biotechnology
funding_total_usd               29940004
status                         operating
country_code                         USA
state_code                            CA
city                           San Mateo
funding_rounds                         3
founded_at           2013-01-01 00:00:00
first_funding_at                     NaN
last_funding_at               09/06/2015
Name: 31357, dtype: object

In [39]:
# and again
companies['first_funding_at'] = pd.to_datetime(companies['first_funding_at'], format='mixed')

In [40]:
companies.info() # finally

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55975 entries, 0 to 55974
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               55975 non-null  object        
 1   category_list      55975 non-null  object        
 2   funding_total_usd  55975 non-null  object        
 3   status             55975 non-null  object        
 4   country_code       55975 non-null  object        
 5   state_code         55975 non-null  object        
 6   city               55975 non-null  object        
 7   funding_rounds     55975 non-null  int64         
 8   founded_at         45266 non-null  datetime64[ns]
 9   first_funding_at   55950 non-null  datetime64[ns]
 10  last_funding_at    55975 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(8)
memory usage: 4.7+ MB


In [41]:
# for last_funding_at now
companies['last_funding_at'] = pd.to_datetime(companies['last_funding_at'], format='mixed')

In [42]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55975 entries, 0 to 55974
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               55975 non-null  object        
 1   category_list      55975 non-null  object        
 2   funding_total_usd  55975 non-null  object        
 3   status             55975 non-null  object        
 4   country_code       55975 non-null  object        
 5   state_code         55975 non-null  object        
 6   city               55975 non-null  object        
 7   funding_rounds     55975 non-null  int64         
 8   founded_at         45266 non-null  datetime64[ns]
 9   first_funding_at   55950 non-null  datetime64[ns]
 10  last_funding_at    55975 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(7)
memory usage: 4.7+ MB


In [43]:
# now that all date columns are datetime types, I can see if founded_at and first_funding_at can be imputed from each other
companies.head()

,Name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,#fame,Media,10000000,operating,IND,16,Mumbai,1,NaT,2015-05-01,2015-05-01
1,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,2014-04-09,2014-01-03,2014-10-14
2,0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,1,2007-01-01,2008-03-19,2008-03-19
3,004 Technologies,Software,-,operating,USA,IL,Champaign,1,2010-01-01,2014-07-24,2014-07-24
4,Ondine Biomedical Inc.,Biotechnology,762851,operating,CAN,BC,Vancouver,2,1997-01-01,2009-11-09,2009-12-21


In [44]:
# testing on index 1
companies['founded_at'][1] - companies['first_funding_at'][1]

Timedelta('96 days 00:00:00')

In [45]:
# that works so I'll get the average timedelta for non null values in these columns
nonnull = companies[companies['founded_at'].notna() & companies['first_funding_at'].notna()]

In [46]:
nonnull.shape

(45245, 11)

In [47]:
(np.abs(nonnull['founded_at'] - nonnull['first_funding_at'])).mean() # average timedelta is 1440 days or about 4 years

Timedelta('1440 days 15:02:24.891148176')

In [48]:
# it doesn't seem like I can accurately impute founded_at from first_funding_at directly, but I can offset by the averge timedelta amount
# I first need to see which direction the time delta tends to go in 
differences = nonnull['founded_at'] - nonnull['first_funding_at']
differences

1          96 days
2        -443 days
3       -1665 days
4       -4695 days
5        -790 days
           ...    
55967   -5113 days
55970   -2220 days
55971     -55 days
55973     488 days
55974    -365 days
Length: 45245, dtype: timedelta64[ns]

In [49]:
differences = differences.reset_index(drop=True)
differences

0          96 days
1        -443 days
2       -1665 days
3       -4695 days
4        -790 days
           ...    
45240   -5113 days
45241   -2220 days
45242     -55 days
45243     488 days
45244    -365 days
Length: 45245, dtype: timedelta64[ns]

In [50]:
zero_delta = pd.to_timedelta(0)
zero_delta

Timedelta('0 days 00:00:00')

In [51]:
differences[0] > zero_delta

True

In [52]:
differences[1] > zero_delta

False

In [53]:
differences > zero_delta

0         True
1        False
2        False
3        False
4        False
         ...  
45240    False
45241    False
45242    False
45243     True
45244    False
Length: 45245, dtype: bool

In [54]:
(differences == zero_delta).sum() # 2476 rows have the same dates for founded_at and first_funding_at

2476

In [55]:
(differences > zero_delta).sum() # 3782 rows have a later founded_at date than first_funding_at date (this seems a bit strange)

3782

In [56]:
(differences < zero_delta).sum() # 38987 rows have an earlier founded_at date than first_funding_at date

38987

In [57]:
# finding the average difference in time for rows where there's an earlier founded_at date than first_funding_at date
timedelta = (np.abs(differences[differences < zero_delta])).mean()
timedelta

Timedelta('1642 days 22:48:34.017493024')

In [58]:
# so I'll make the null founded_at values equal to the first_funding_at date - timedelta
# first I need to get rid of the rows where both founded_at and first_funding_at are missing
companies.dropna(subset=['founded_at', 'first_funding_at'], how='all', inplace=True)

In [59]:
# resetting the index again to account for dropped rows
companies.reset_index(drop=True, inplace=True)

In [60]:
type(companies['founded_at'][0])

pandas._libs.tslibs.nattype.NaTType

In [61]:
# testing that this would work 
type(companies['founded_at'][0]) == pd._libs.tslibs.nattype.NaTType

True

In [62]:
# imputing the values now
for i in range(len(companies['founded_at'])):
    if type(companies.at[i, 'founded_at']) == pd._libs.tslibs.nattype.NaTType:
        companies.at[i, 'founded_at'] = companies.at[i, 'first_funding_at'] - timedelta

In [63]:
# only the first_funding_at column has any nulls now
companies.isna().sum()

Name                  0
category_list         0
funding_total_usd     0
status                0
country_code          0
state_code            0
city                  0
funding_rounds        0
founded_at            0
first_funding_at     21
last_funding_at       0
dtype: int64

In [64]:
companies.reset_index(drop=True, inplace=True)

In [65]:
# imputing those values from founded_at column by adding in the timedelta
for i in range(len(companies['first_funding_at'])):
    if type(companies.at[i, 'first_funding_at']) == pd._libs.tslibs.nattype.NaTType:
        companies.at[i, 'first_funding_at'] = companies.at[i, 'founded_at'] + timedelta

In [66]:
companies.isna().sum()

Name                 0
category_list        0
funding_total_usd    0
status               0
country_code         0
state_code           0
city                 0
funding_rounds       0
founded_at           0
first_funding_at     0
last_funding_at      0
dtype: int64

In [67]:
# there are no more nulls to deal with
# but there are other issues
# first quickly check for duplicates
companies.duplicated().sum()

0

In [68]:
# I can see that there's a row with a '-' in the funding_total_usd column
companies.head() 

,Name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,#fame,Media,10000000,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01
1,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14
2,0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19
3,004 Technologies,Software,-,operating,USA,IL,Champaign,1,2010-01-01 00:00:00.000000000,2014-07-24,2014-07-24
4,Ondine Biomedical Inc.,Biotechnology,762851,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21


In [69]:
# and the column is of type object not int or float
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55971 entries, 0 to 55970
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               55971 non-null  object        
 1   category_list      55971 non-null  object        
 2   funding_total_usd  55971 non-null  object        
 3   status             55971 non-null  object        
 4   country_code       55971 non-null  object        
 5   state_code         55971 non-null  object        
 6   city               55971 non-null  object        
 7   funding_rounds     55971 non-null  int64         
 8   founded_at         55971 non-null  datetime64[ns]
 9   first_funding_at   55971 non-null  datetime64[ns]
 10  last_funding_at    55971 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(7)
memory usage: 4.7+ MB


In [70]:
# how many rows are there with a '-'
companies[companies['funding_total_usd'] == '-'].shape # there are 9010 rows with a '-'

(9010, 11)

In [71]:
# this is a lot of rows to drop, but I will for now since it doesn't make sense to impute values for a target column
# first get the indeces for the rows I want to drop
dash_index = companies[companies['funding_total_usd'] == '-'].index

In [72]:
companies.drop(labels=dash_index, inplace=True)

In [73]:
companies.head()

,Name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,#fame,Media,10000000,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01
1,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14
2,0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19
4,Ondine Biomedical Inc.,Biotechnology,762851,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21
5,H2O.ai,Analytics,33600000,operating,USA,CA,Mountain View,4,2011-01-01 00:00:00.000000000,2013-03-01,2015-09-11


In [74]:
# need to reset index again
companies.reset_index(drop=True, inplace=True)

In [75]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46961 entries, 0 to 46960
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               46961 non-null  object        
 1   category_list      46961 non-null  object        
 2   funding_total_usd  46961 non-null  object        
 3   status             46961 non-null  object        
 4   country_code       46961 non-null  object        
 5   state_code         46961 non-null  object        
 6   city               46961 non-null  object        
 7   funding_rounds     46961 non-null  int64         
 8   founded_at         46961 non-null  datetime64[ns]
 9   first_funding_at   46961 non-null  datetime64[ns]
 10  last_funding_at    46961 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(7)
memory usage: 3.9+ MB


In [76]:
# I just need to change the type of the column now
companies['funding_total_usd'] = companies['funding_total_usd'].astype(float)

In [77]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46961 entries, 0 to 46960
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               46961 non-null  object        
 1   category_list      46961 non-null  object        
 2   funding_total_usd  46961 non-null  float64       
 3   status             46961 non-null  object        
 4   country_code       46961 non-null  object        
 5   state_code         46961 non-null  object        
 6   city               46961 non-null  object        
 7   funding_rounds     46961 non-null  int64         
 8   founded_at         46961 non-null  datetime64[ns]
 9   first_funding_at   46961 non-null  datetime64[ns]
 10  last_funding_at    46961 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(1), object(6)
memory usage: 3.9+ MB


In [78]:
# moving on to the category_list column and cleaning that up
companies['category_list'].value_counts()

category_list
Software                                                                                                          3305
Biotechnology                                                                                                     3290
E-Commerce                                                                                                         928
Clean Technology                                                                                                   901
Mobile                                                                                                             887
                                                                                                                  ... 
Fashion|Search                                                                                                       1
Consumer Electronics|Hardware|Hardware + Software|Internet TV|Software                                               1
Business Development|Business Inte

In [79]:
# there are 20,593 unique values in the category_list column, but these are really lists of categories so I will separate them into individual categories
# creating a new column that is a list of the categories from category_list
companies['categories'] = companies['category_list'].apply(lambda x: x.split('|'))

In [80]:
companies.head()

,Name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,categories
0,#fame,Media,10000000.0,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01,[Media]
1,:Qounter,Application Platforms|Real Time|Social Network...,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,"[Application Platforms, Real Time, Social Netw..."
2,0-6.com,Curated Web,2000000.0,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19,[Curated Web]
3,Ondine Biomedical Inc.,Biotechnology,762851.0,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21,[Biotechnology]
4,H2O.ai,Analytics,33600000.0,operating,USA,CA,Mountain View,4,2011-01-01 00:00:00.000000000,2013-03-01,2015-09-11,[Analytics]


In [81]:
# dropping the category_list column since I've replaced it
companies.drop(columns='category_list', inplace=True)

In [82]:
companies.explode('categories') # I can use this to get the count of unique categories

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,categories
0,#fame,10000000.0,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01,Media
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,Application Platforms
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,Real Time
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,Social Network Media
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19,Curated Web
...,...,...,...,...,...,...,...,...,...,...,...
46959,Zzzzapp Wireless ltd.,114304.0,operating,HRV,15,Split,4,2012-05-13 00:00:00.000000000,2011-01-11,2014-01-03,Web Development
46959,Zzzzapp Wireless ltd.,114304.0,operating,HRV,15,Split,4,2012-05-13 00:00:00.000000000,2011-01-11,2014-01-03,Wireless
46960,Ôasys,18192.0,operating,USA,CA,San Francisco,1,2014-01-01 00:00:00.000000000,2015-01-01,2015-01-01,Consumer Electronics
46960,Ôasys,18192.0,operating,USA,CA,San Francisco,1,2014-01-01 00:00:00.000000000,2015-01-01,2015-01-01,Internet of Things


In [83]:
companies.explode('categories')['categories'].value_counts() # got the unique count of categories down from 20,593 to 850

categories
Software               6957
Mobile                 4227
Biotechnology          4111
E-Commerce             3016
Enterprise Software    2252
                       ... 
Made in Italy             1
Private Corrections       1
Hi Tech                   1
Hip Hop                   1
Micro-Enterprises         1
Name: count, Length: 850, dtype: int64

# Preprocessing <a class="anchor" id="second-bullet"></a>

Now that the data has been cleaned I need to get it ready for modelling. I'll continue with the categories column to start. I want to create dummy columns for some number of the top categories.

In [84]:
# What percentage of the data do 50 categories cover?
companies.explode('categories')['categories'].value_counts(normalize = True)[:50].sum()

0.5594293521097157

55% seems good enough for now, I'll dummy those 50 categories and lump the rest under 'Other'

In [85]:
# getting a list of these top 50 categories
list(companies.explode('categories')['categories'].value_counts()[:50].index)

['Software',
 'Mobile',
 'Biotechnology',
 'E-Commerce',
 'Enterprise Software',
 'Curated Web',
 'Social Media',
 'Health Care',
 'Advertising',
 'SaaS',
 'Health and Wellness',
 'Games',
 'Analytics',
 'Education',
 'Internet',
 'Finance',
 'Technology',
 'Hardware + Software',
 'Clean Technology',
 'Apps',
 'Manufacturing',
 'Services',
 'Security',
 'Marketplaces',
 'Video',
 'Medical',
 'Big Data',
 'Cloud Computing',
 'Fashion',
 'Travel',
 'Search',
 'Information Technology',
 'Networking',
 'Sales and Marketing',
 'Music',
 'Semiconductors',
 'Social Network Media',
 'Real Estate',
 'Consulting',
 'Entertainment',
 'Startups',
 'News',
 'Media',
 'FinTech',
 'Web Hosting',
 'Retail',
 'Financial Services',
 'Sports',
 'Hospitality',
 'Messaging']

In [86]:
# creating a copy of the companies dataframe to test this out
df = companies.copy()

In [87]:
df.head()

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,categories
0,#fame,10000000.0,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01,[Media]
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,"[Application Platforms, Real Time, Social Netw..."
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19,[Curated Web]
3,Ondine Biomedical Inc.,762851.0,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21,[Biotechnology]
4,H2O.ai,33600000.0,operating,USA,CA,Mountain View,4,2011-01-01 00:00:00.000000000,2013-03-01,2015-09-11,[Analytics]


In [88]:
# creating the dummy columns and populating them based on the categories column - 
# the Other column gets populated for any row that does not already had a top category column checked off
top_categories = list(df.explode('categories')['categories'].value_counts()[:50].index)
top_categories.sort()
for category in top_categories:
    df[category] = df.apply(lambda row: 1 if category in row['categories'] else 0, axis=1)
df['Other'] = df.apply(lambda row: 1 if len(np.setdiff1d(row['categories'], top_categories)) == len(row['categories']) else 0, axis=1)

In [89]:
pd.set_option('display.max_columns', 500)

In [90]:
df.head()

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,categories,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other
0,#fame,10000000.0,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01,[Media],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,"[Application Platforms, Real Time, Social Netw...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19,[Curated Web],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ondine Biomedical Inc.,762851.0,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21,[Biotechnology],0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H2O.ai,33600000.0,operating,USA,CA,Mountain View,4,2011-01-01 00:00:00.000000000,2013-03-01,2015-09-11,[Analytics],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [91]:
# this seems to have worked, I'll now delete the categories column
df.drop(columns='categories', inplace=True)

Now that I've dummied the categories column, I'll move on to the date columns. First I will change founded_at to the age of the company, using the latest date in the dataset as the present time since I'm not sure when this dataset was created. Then I'll change first_funding_at to the difference in time between it and the company age, which would represent the time to first funding from when the company started (will be negative in some instances). Lastly I'll change last_funding_at to the difference in time between it and the time of first funding.

In [92]:
# checking the max value in the founded_at column
df['founded_at'].max()

Timestamp('2104-01-01 00:00:00')

In [93]:
# that doesn't look right, which row is that
df.loc[df['founded_at']=='2104-01-01 00:00:00']

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other
37871,Soply,150000.0,operating,GBR,H9,London,1,2104-01-01,2013-01-07,2013-01-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
# I'm going to assume the date was inputted incorrectly and was intended to be 2014, so I'll change it to that
df.at[37871, 'founded_at'] = '2014-01-01 00:00:00'

In [95]:
# checking that it was changed
df.at[37871, 'founded_at']

Timestamp('2014-01-01 00:00:00')

In [96]:
# what's the max now?
df['founded_at'].max()

Timestamp('2015-12-23 00:00:00')

In [97]:
# that seems pretty normal, I'll just double check that it makes sense in context
df.loc[df['founded_at']=='2015-12-23 00:00:00']

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other
2093,American Development Group,5300000.0,operating,USA,NY,Brooklyn,1,2015-12-23,2015-09-01,2015-09-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [98]:
# that looks good, I'll check the max in first_funding_at now
df['first_funding_at'].max()

Timestamp('2019-10-01 22:48:34.017493024')

In [99]:
# checking if that makes sense
df.loc[df['first_funding_at']=='2019-10-01 22:48:34.017493024']

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other
12570,EKU Power Drives,297149.0,operating,DEU,1,Stuttgart,3,2015-04-03,2019-10-01 22:48:34.017493024,2015-01-20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [100]:
# that doesn't look right, I'll change it to the same date as is in last_funding_at
df.at[12570, 'first_funding_at'] = df.at[12570, 'last_funding_at']

In [101]:
# checking
df.at[12570, 'first_funding_at']

Timestamp('2015-01-20 00:00:00')

In [102]:
# what's the new max 
df['first_funding_at'].max()

Timestamp('2019-07-02 22:48:34.017493024')

In [103]:
df.loc[df['first_funding_at']=='2019-07-02 22:48:34.017493024']

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other
16714,"Golfler Holdings, LLC",272500.0,operating,USA,MI,Detroit,4,2015-01-02,2019-07-02 22:48:34.017493024,2015-06-14,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


In [104]:
# that doesn't look right either, I'll change it to the last_funding_at value
df.at[16714, 'first_funding_at'] = df.at[16714, 'last_funding_at']

In [105]:
# checking the new max
df['first_funding_at'].max()

Timestamp('2019-06-04 22:48:34.017493024')

In [106]:
# another one in 2019 which I'm guessing is also an issue. I'll just write a loop to go through df and correct any of these particular issues
df.shape 
for i in range(46961):
    if df.at[i, 'first_funding_at'] > df.at[i, 'last_funding_at']:
        df.at[i, 'first_funding_at'] = df.at[i, 'last_funding_at']

In [107]:
# what's the max now?
df['first_funding_at'].max()

Timestamp('2015-12-11 00:00:00')

In [108]:
df.loc[df['first_funding_at']=='2015-12-11 00:00:00']

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other
6314,Building Energy,6.300000e+05,operating,USA,OR,Portland,1,2011-06-12 01:11:25.982506976,2015-12-11,2015-12-11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6513,C-Labs Corporation,2.700000e+06,operating,USA,WA,Redmond,1,2009-05-28 00:00:00.000000000,2015-12-11,2015-12-11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8852,Cola,1.300000e+06,operating,USA,CA,San Francisco,1,2015-01-01 00:00:00.000000000,2015-12-11,2015-12-11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16509,Globavir,1.500000e+05,operating,USA,CA,Los Altos,1,2014-01-01 00:00:00.000000000,2015-12-11,2015-12-11,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18544,Huuuge Games,4.000000e+06,operating,USA,CA,Palo Alto,1,2014-01-01 00:00:00.000000000,2015-12-11,2015-12-11,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18754,Iceye,2.800000e+06,operating,FIN,13,Espoo,1,2012-01-01 00:00:00.000000000,2015-12-11,2015-12-11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24035,MAKEUS,1.740000e+07,operating,JPN,18,China,1,2011-06-12 01:11:25.982506976,2015-12-11,2015-12-11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
33041,Qvivr,1.755027e+06,operating,USA,CA,Santa Clara,1,2014-01-01 00:00:00.000000000,2015-12-11,2015-12-11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
33490,Recardio,3.000000e+06,closed,USA,CA,San Francisco,1,2011-06-12 01:11:25.982506976,2015-12-11,2015-12-11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
34141,Reusable Solutions Group,2.500000e+05,operating,USA,CA,Santa Cruz,1,2011-06-12 01:11:25.982506976,2015-12-11,2015-12-11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [109]:
# alright that seems believable now, let's check the max for last_funding at
df['last_funding_at'].max()

Timestamp('2105-01-05 00:00:00')

In [110]:
# looks like another typo, I'll change it manually
df.loc[df['last_funding_at']=='2105-01-05 00:00:00']
df.at[33288, 'last_funding_at'] = '2015-01-05 00:00:00'

In [111]:
df.at[33288, 'last_funding_at']

Timestamp('2015-01-05 00:00:00')

In [112]:
# what's the max now?
df['last_funding_at'].max()

Timestamp('2015-12-12 00:00:00')

In [113]:
# ok so the max value from all the columns is 2015-12-23, I'll use that as the present date
present = df['founded_at'].max()

In [114]:
# checking that the minimum value of all founded dates subtracted from the present day is 0 (or higher)
(present - df['founded_at']).min()

Timedelta('0 days 00:00:00')

In [115]:
# checking that I can get just the days as ints from this new series of timedeltas
(present - df['founded_at']).dt.days

0        1878
1         623
2        3278
3        6930
4        1817
         ... 
46956    3278
46957     694
46958    3091
46959    1319
46960     721
Name: founded_at, Length: 46961, dtype: int64

In [116]:
# creating the new column with the integer value in days
df['company_age_days'] = (present - df['founded_at']).dt.days

In [117]:
df.head()

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days
0,#fame,10000000.0,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278
3,Ondine Biomedical Inc.,762851.0,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930
4,H2O.ai,33600000.0,operating,USA,CA,Mountain View,4,2011-01-01 00:00:00.000000000,2013-03-01,2015-09-11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817


In [118]:
# creating the new column for the amount of time from founding to funding
df['founding_to_funding_days'] = (df['first_funding_at'] - df['founded_at']).dt.days

In [119]:
df.head()

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days
0,#fame,10000000.0,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443
3,Ondine Biomedical Inc.,762851.0,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695
4,H2O.ai,33600000.0,operating,USA,CA,Mountain View,4,2011-01-01 00:00:00.000000000,2013-03-01,2015-09-11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790


In [120]:
# creating the new column for the amount of time from first funding to last
df['first_funding_to_last'] = (df['last_funding_at'] - df['first_funding_at']).dt.days

In [121]:
df.head()

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last
0,#fame,10000000.0,operating,IND,16,Mumbai,1,2010-10-31 01:11:25.982506976,2015-05-01,2015-05-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642,0
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,2014-04-09 00:00:00.000000000,2014-01-03,2014-10-14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96,284
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,2007-01-01 00:00:00.000000000,2008-03-19,2008-03-19,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443,0
3,Ondine Biomedical Inc.,762851.0,operating,CAN,BC,Vancouver,2,1997-01-01 00:00:00.000000000,2009-11-09,2009-12-21,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695,42
4,H2O.ai,33600000.0,operating,USA,CA,Mountain View,4,2011-01-01 00:00:00.000000000,2013-03-01,2015-09-11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790,924


In [122]:
# now that I've created those three columns I can remove the original three datetime columns
df.drop(columns=['founded_at', 'first_funding_at', 'last_funding_at'], inplace=True)

In [123]:
df.head()

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last
0,#fame,10000000.0,operating,IND,16,Mumbai,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642,0
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96,284
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443,0
3,Ondine Biomedical Inc.,762851.0,operating,CAN,BC,Vancouver,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695,42
4,H2O.ai,33600000.0,operating,USA,CA,Mountain View,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790,924


Now that I've dealt with the date columns I can move on to processing the location columns. I want to first see what the value counts/proportions are for the different values in those columns to determine which dummy columns to make. I will probably drop the state_code column and just use the city column since doing both seems redundant and if I'm using cities then the states can be inferred.

In [124]:
df['country_code'].value_counts()

country_code
USA    31015
GBR     2929
CAN     1553
IND     1151
CHN     1088
       ...  
HND        1
BWA        1
TGO        1
ALB        1
PRY        1
Name: count, Length: 116, dtype: int64

In [125]:
# I can see right away that USA accounts for the largest chunk for any one country by far, let's see the proportion
df['country_code'].value_counts(normalize=True)

country_code
USA    0.660442
GBR    0.062371
CAN    0.033070
IND    0.024510
CHN    0.023168
         ...   
HND    0.000021
BWA    0.000021
TGO    0.000021
ALB    0.000021
PRY    0.000021
Name: proportion, Length: 116, dtype: float64

In [126]:
# USA accounts for 66% of the data. Since this is such a high proportion I'll just create two dummy columns (or one) for this - USA vs. not USA
df['country_is_USA'] = np.where(df['country_code'] == 'USA', 1, 0)

In [127]:
df.head()

,Name,funding_total_usd,status,country_code,state_code,city,funding_rounds,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last,country_is_USA
0,#fame,10000000.0,operating,IND,16,Mumbai,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642,0,0
1,:Qounter,700000.0,operating,USA,DE,Delaware City,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96,284,1
2,0-6.com,2000000.0,operating,CHN,22,Beijing,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443,0,0
3,Ondine Biomedical Inc.,762851.0,operating,CAN,BC,Vancouver,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695,42,0
4,H2O.ai,33600000.0,operating,USA,CA,Mountain View,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790,924,1


In [128]:
# moving on to the state_code and city columns
df.drop(columns='state_code', inplace=True)

In [129]:
df['city'].value_counts()

city
San Francisco    2970
New York         2684
London           1571
Seattle           676
Austin            660
                 ... 
Lambertville        1
Celbridge           1
Flemington          1
Phoenixville        1
Split               1
Name: count, Length: 4228, dtype: int64

In [130]:
df['city'].value_counts(normalize=True)

city
San Francisco    0.063244
New York         0.057154
London           0.033453
Seattle          0.014395
Austin           0.014054
                   ...   
Lambertville     0.000021
Celbridge        0.000021
Flemington       0.000021
Phoenixville     0.000021
Split            0.000021
Name: proportion, Length: 4228, dtype: float64

In [131]:
# how many cities account for at least 1% of the data?
(df['city'].value_counts(normalize=True) >= .01).sum()

14

In [132]:
list(df['city'].value_counts()[:14].index)

['San Francisco',
 'New York',
 'London',
 'Seattle',
 'Austin',
 'Palo Alto',
 'Cambridge',
 'Boston',
 'Los Angeles',
 'Chicago',
 'San Diego',
 'Mountain View',
 'Paris',
 'Beijing']

In [133]:
# I will create columns for these 14 cities
top_cities = list(df['city'].value_counts()[:14].index)
top_cities.sort()
for city in top_cities:
    df[city] = df.apply(lambda row: 1 if city == row['city'] else 0, axis=1)

In [134]:
df.head()

,Name,funding_total_usd,status,country_code,city,funding_rounds,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last,country_is_USA,Austin,Beijing,Boston,Cambridge,Chicago,London,Los Angeles,Mountain View,New York,Palo Alto,Paris,San Diego,San Francisco,Seattle
0,#fame,10000000.0,operating,IND,Mumbai,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,:Qounter,700000.0,operating,USA,Delaware City,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96,284,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0-6.com,2000000.0,operating,CHN,Beijing,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Ondine Biomedical Inc.,762851.0,operating,CAN,Vancouver,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H2O.ai,33600000.0,operating,USA,Mountain View,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790,924,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [135]:
# dropping the country_code and city columns now
df.drop(columns=['country_code', 'city'], inplace=True)

In [136]:
df.head()

,Name,funding_total_usd,status,funding_rounds,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last,country_is_USA,Austin,Beijing,Boston,Cambridge,Chicago,London,Los Angeles,Mountain View,New York,Palo Alto,Paris,San Diego,San Francisco,Seattle
0,#fame,10000000.0,operating,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,:Qounter,700000.0,operating,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96,284,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0-6.com,2000000.0,operating,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Ondine Biomedical Inc.,762851.0,operating,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H2O.ai,33600000.0,operating,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790,924,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


Now I need to create my y column, which will be based on the status and funding_total_usd columns. A company will be considered a success if it has IPO'd, been acquired, or is still operating and is above a certain threshold of funding total. A company will be considered a failure if it has closed or is still operating but has raised below a certain threshold. Now I need to determine what that threshold should be.

In [137]:
df['status'].value_counts()

status
operating    37500
acquired      4519
closed        3562
ipo           1380
Name: count, dtype: int64

In [138]:
# of the operating businesses, what is the average amount raised?
df[df['status']=='operating']['funding_total_usd'].mean() # 14,084,112.60

14084112.601714384

In [139]:
# what's the median?
df[df['status']=='operating']['funding_total_usd'].median() # 1,700,000.00

1700000.0

In [140]:
# how many companies raised at least 1.7 million?
(df[df['status']=='operating']['funding_total_usd'] >= 1700000.0).sum()

18774

If 1.7 million is my cutoff that would mean there would be 18,774 + 4,519 + 1,380 = 24,673 successful companies in the dataset, and 18,726 + 3,562 = 22,288 failed companies. This is 53% and 47% respectively, which is pretty close to even. I'll make \$1.7 million my threshold.

In [141]:
# creating a success column using the above stated conditions
df['successful'] = np.where((df['status'] == 'ipo') | (df['status'] == 'acquired') | ((df['status'] == 'operating') & (df['funding_total_usd'] >= 1700000.0)), 1, 0)

In [142]:
df.head()

,Name,funding_total_usd,status,funding_rounds,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last,country_is_USA,Austin,Beijing,Boston,Cambridge,Chicago,London,Los Angeles,Mountain View,New York,Palo Alto,Paris,San Diego,San Francisco,Seattle,successful
0,#fame,10000000.0,operating,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,:Qounter,700000.0,operating,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96,284,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0-6.com,2000000.0,operating,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Ondine Biomedical Inc.,762851.0,operating,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H2O.ai,33600000.0,operating,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790,924,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [143]:
# saving the name column in a separate variable
name = df['Name']

In [144]:
# dropping the status, funding_total_usd, and Name columns
df.drop(columns=['status', 'funding_total_usd', 'Name'], inplace=True)

In [145]:
df.head()

,funding_rounds,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last,country_is_USA,Austin,Beijing,Boston,Cambridge,Chicago,London,Los Angeles,Mountain View,New York,Palo Alto,Paris,San Diego,San Francisco,Seattle,successful
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1878,1642,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,623,-96,284,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3278,443,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6930,4695,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1817,790,924,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


# Modelling <a class="anchor" id="third-bullet"></a>

In [146]:
X = df.drop(columns='successful')
y = df['successful']

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


#Fit a logistic regression model
logreg_model = LogisticRegression(max_iter=10000)
logreg_model.fit(X_train, y_train)

#Score it
train_prediction = logreg_model.predict(X_train)
test_prediction = logreg_model.predict(X_test)
accuracy_train = accuracy_score(train_prediction, y_train)
accuracy_test = accuracy_score(test_prediction, y_test)

print(f"Score on training set: {accuracy_train}")
print(f"Score on test set: {accuracy_test}")


Score on training set: 0.7186663421757119
Score on test set: 0.7121868124068422


In [148]:
pd.DataFrame(logreg_model.coef_,  columns = X.columns)

,funding_rounds,Advertising,Analytics,Apps,Big Data,Biotechnology,Clean Technology,Cloud Computing,Consulting,Curated Web,E-Commerce,Education,Enterprise Software,Entertainment,Fashion,FinTech,Finance,Financial Services,Games,Hardware + Software,Health Care,Health and Wellness,Hospitality,Information Technology,Internet,Manufacturing,Marketplaces,Media,Medical,Messaging,Mobile,Music,Networking,News,Real Estate,Retail,SaaS,Sales and Marketing,Search,Security,Semiconductors,Services,Social Media,Social Network Media,Software,Sports,Startups,Technology,Travel,Video,Web Hosting,Other,company_age_days,founding_to_funding_days,first_funding_to_last,country_is_USA,Austin,Beijing,Boston,Cambridge,Chicago,London,Los Angeles,Mountain View,New York,Palo Alto,Paris,San Diego,San Francisco,Seattle
0,0.353442,-0.007327,0.043858,-0.194392,0.06038,0.343662,0.08595,0.031881,-0.088029,-0.319082,-0.285391,-0.258642,0.165366,-0.060802,-0.073732,0.04615,0.040407,0.075123,-0.160307,-0.042786,0.123205,-0.058589,-0.09676,0.034325,-0.123834,-0.053715,-0.180129,-0.033038,0.023404,-0.044582,-0.227863,-0.136553,-0.047961,-0.049989,-0.073041,-0.012158,-0.114186,-0.038509,-0.088736,0.102138,0.123313,-0.037802,-0.440718,-0.133841,-0.117926,-0.135774,-0.082491,-0.052142,-0.09019,-0.04159,0.039189,-0.198205,0.000253,0.000004,0.000892,0.279546,0.015246,0.160451,0.05047,0.09988,0.017887,-0.210944,-0.029323,0.129288,0.137331,0.083844,0.038154,0.043818,0.31446,0.033146


In [149]:
np.exp(logreg_model.coef_)

array([[1.4239607 , 0.99269978, 1.04483383, 0.82333542, 1.06224057,
        1.41010166, 1.08975136, 1.03239476, 0.91573408, 0.72681567,
        0.75172008, 0.77209914, 1.17982454, 0.94100967, 0.92892038,
        1.0472313 , 1.04123437, 1.07801626, 0.8518818 , 0.95811671,
        1.13111583, 0.94309437, 0.90777348, 1.03492101, 0.88352615,
        0.94770253, 0.83516221, 0.96750216, 1.02368034, 0.95639717,
        0.79623347, 0.87236   , 0.95317064, 0.95123954, 0.92956237,
        0.98791543, 0.89209208, 0.96222335, 0.91508698, 1.10753585,
        1.13123869, 0.96290346, 0.64357443, 0.8747288 , 0.88876157,
        0.87304008, 0.92081979, 0.94919404, 0.91375798, 0.95926324,
        1.03996693, 0.82020156, 1.00025267, 1.00000387, 1.00089191,
        1.32252897, 1.01536282, 1.1740408 , 1.05176538, 1.10503823,
        1.01804822, 0.80981923, 0.9711023 , 1.13801798, 1.14720838,
        1.08745877, 1.03889084, 1.04479225, 1.36951922, 1.03370112]])